In [3]:
import pysam
import os
os.chdir('/SGRNJ06/randd/USER/zhouyiqi/work/analysis/mobiuscope/coverage/BAM')

In [5]:
from collections import defaultdict


def nested_defaultdict(dim=3, val_type=int):
    if dim == 1:
        return defaultdict(val_type)
    else:
        return defaultdict(lambda: nested_defaultdict(dim - 1, val_type=val_type))

In [7]:

bam = './NB4_K562_subset.bam'
dup_dict = nested_defaultdict(dim=4)
removed = 0
with pysam.AlignmentFile(bam, "rb") as reader:
    header = reader.header.to_dict()
    out_bam_file = 'UMI.bam'
    with pysam.AlignmentFile(out_bam_file, "wb", header=header) as writer:
        for record in reader:
            gn = record.get_tag("GN")
            cb = record.get_tag("CB")
            umi = record.get_tag("UB")
            if any(x == "-" for x in (gn, cb, umi)):
                continue
            rn, rs = record.reference_name, record.reference_start
            dup_dict[cb][rn][rs][umi] += 1
            if dup_dict[cb][rn][rs][umi] > 1:
                removed += 1
                continue
            writer.write(record)

[E::idx_find_and_load] Could not retrieve index file for './NB4_K562_subset.bam'


In [8]:
removed

2621354